### Boid Simulation with Gradient Color Change

In [145]:
import math
import random
import pygame
import sys


# Size of canvas. These get updated to fill the whole screen.
width = 1000
height = 1000
boidneighbor = 5
numBoids = 100
visualRange = 75


boids = []


def initBoids():
   global boids
   boids = []
   for i in range(numBoids):
       boids.append({
           'x': random.random() * width,
           'y': random.random() * height,
           'dx': random.random() * 10 - 5,
           'dy': random.random() * 10 - 5,
           'history': [],
       })


def distance(boid1, boid2):
   return math.sqrt((boid1['x'] - boid2['x'])**2 + (boid1['y'] - boid2['y'])**2)


def nClosestBoids(boid, n):
   sorted_boids = sorted(boids, key=lambda other_boid: distance(boid, other_boid))
   return sorted_boids[1:n+1]


def sizeCanvas():
   global width, height
   size = (width, height)
   return pygame.display.set_mode(size)


def keepWithinBounds(boid):
   margin = 200
   turnFactor = 1


   if boid['x'] < margin:
       boid['dx'] += turnFactor
   if boid['x'] > width - margin:
       boid['dx'] -= turnFactor
   if boid['y'] < margin:
       boid['dy'] += turnFactor
   if boid['y'] > height - margin:
       boid['dy'] -= turnFactor


def flyTowardsCenter(boid):
   centeringFactor = 0.005
   neighbormin = 3
   centerX = 0
   centerY = 0
   numNeighbors = 0


   for otherBoid in boids:
       if distance(boid, otherBoid) < visualRange:
           centerX += otherBoid['x']
           centerY += otherBoid['y']
           numNeighbors += 1
      
   if numNeighbors <= neighbormin:
       for otherBoid in boids:
           if otherBoid in nClosestBoids(boid, boidneighbor):
               centerX += otherBoid['x']
               centerY += otherBoid['y']
               numNeighbors += 1
           if numNeighbors == neighbormin:
               break
                 
   if numNeighbors:
       centerX /= numNeighbors
       centerY /= numNeighbors


       boid['dx'] += (centerX - boid['x']) * centeringFactor
       boid['dy'] += (centerY - boid['y']) * centeringFactor


def avoidOthers(boid):
   minDistance = 20
   avoidFactor = 0.05
   moveX = 0
   moveY = 0


   for otherBoid in boids:
       if otherBoid != boid:
           if distance(boid, otherBoid) < minDistance:
               moveX += boid['x'] - otherBoid['x']
               moveY += boid['y'] - otherBoid['y']


   boid['dx'] += moveX * avoidFactor
   boid['dy'] += moveY * avoidFactor


def matchVelocity(boid):
   matchingFactor = 0.05
   neighbormin = 3
   avgDX = 0
   avgDY = 0
   numNeighbors = 0


   for otherBoid in boids:
       if distance(boid, otherBoid) < visualRange:
           avgDX += otherBoid['dx']
           avgDY += otherBoid['dy']
           numNeighbors += 1


   if numNeighbors <= neighbormin:
       for otherBoid in boids:
           if otherBoid in nClosestBoids(boid, boidneighbor):
               avgDX += otherBoid['dx']
               avgDY += otherBoid['dy']
               numNeighbors += 1
           if numNeighbors == neighbormin:
               break


   if numNeighbors:
       avgDX /= numNeighbors
       avgDY /= numNeighbors


       boid['dx'] += (avgDX - boid['dx']) * matchingFactor
       boid['dy'] += (avgDY - boid['dy']) * matchingFactor


def limitSpeed(boid):
   speedLimit = 15


   speed = math.sqrt(boid['dx']**2 + boid['dy']**2)
   if speed > speedLimit:
       boid['dx'] = (boid['dx'] / speed) * speedLimit
       boid['dy'] = (boid['dy'] / speed) * speedLimit

def emo_boid():
    emo_boid_num = 1
    emo_boid_lis = []
    for i in range (emo_boid_num):
        emo_boid_lis.append(random.choice(boids))
    for emo_boid in emo_boid_lis:
        def avoidOthers():
            minDistance = 20
            avoidFactor = 0.05
            moveX = 0
            moveY = 0

            for otherBoid in boids:
                if otherBoid != boid:
                    if distance(boid, otherBoid) < minDistance:
                        moveX += boid['x'] - otherBoid['x']
                        moveY += boid['y'] - otherBoid['y']

    boid['dx'] += moveX * avoidFactor
    boid['dy'] += moveY * avoidFactor   

def drawBoid(screen, boid, color):
   angle = math.atan2(boid['dy'], boid['dx'])
  
   # Define the arrow shape
   arrow_shape = [
       (0, -15),   # tip of the arrow
       (10, -5),   # right wing
       (10, 0),    # right wing end
       (20, 0),    # tail end
       (10, 5),    # left wing end
       (10, 5),    # left wing
       (0, 15)     # tip of the arrow
   ]
  
   # Rotate the points
   rotated_points = []
   for point in arrow_shape:
       x, y = point
       rotated_x = x * math.cos(angle) - y * math.sin(angle)
       rotated_y = x * math.sin(angle) + y * math.cos(angle)
       rotated_points.append((rotated_x + boid['x'], rotated_y + boid['y']))
  
   # Draw the arrow shape
   pygame.draw.polygon(screen, color, rotated_points)
  
   if DRAW_TRAIL:
       # Draw trail with fading effect
       trail_length = len(boid['history'])
       for i, point in enumerate(boid['history']):
           # Fading effect: older points are more transparent
           alpha = int(255 * (1 - (i / trail_length)) * 0.5)  # Adjust fading speed here
           faded_color = (color[0], color[1], color[2], alpha)
           s = pygame.Surface((4, 4), pygame.SRCALPHA)
           pygame.draw.circle(s, faded_color, (2, 2), 2)
           screen.blit(s, (int(point[0]), int(point[1])))


# Main animation loop
def animationLoop():
   global boids
   screen.fill((0, 0, 0)) # Background color


   for boid in boids:
       flyTowardsCenter(boid)
       avoidOthers(boid)
       matchVelocity(boid)
       limitSpeed(boid)
       keepWithinBounds(boid)


       boid['x'] += boid['dx']
       boid['y'] += boid['dy']
       boid['history'].append((boid['x'], boid['y']))
       boid['history'] = boid['history'][-30:] # Limit history length to shorter trail


       # Determine color based on the number of neighbors
       numNeighbors = sum(distance(boid, otherBoid) < visualRange for otherBoid in boids)
       brightness = min(255, numNeighbors * 10)  # Adjust brightness scaling if needed
       if numNeighbors > 3:
           # Blend from blue to yellow
           red = 255
           green = brightness
           blue = 0
       else:
           # Blue when alone or with few neighbors
           red = 0
           green = 0
           blue = 255
      
       color = (red, green, blue)  # RGB color transitioning from blue to yellow


       drawBoid(screen, boid, color)


   pygame.display.flip()
   pygame.time.Clock().tick(60)


pygame.init()
screen = sizeCanvas()
DRAW_TRAIL = True


initBoids()
while True:
   for event in pygame.event.get():
       if event.type == pygame.QUIT:
           pygame.quit()
           sys.exit()
   animationLoop()

KeyboardInterrupt: 

### Add Perlin Noise

In [148]:
import math
import random
import pygame
import sys
import noise
import numpy as np

# Size of canvas. These get updated to fill the whole screen.
width = 1400
height = 800
boidneighbor = 5
numBoids = 100
visualRange = 75
DRAW_TRAIL = True
z_offset = 0

boids = []

def initBoids():
    global boids
    boids = []
    for i in range(numBoids):
        boids.append({
            'x': random.random() * width,
            'y': random.random() * height,
            'dx': random.random() * 10 - 5,
            'dy': random.random() * 10 - 5,
            'history': [],
        })

def distance(boid1, boid2):
    return math.sqrt((boid1['x'] - boid2['x'])**2 + (boid2['y'] - boid1['y'])**2)

def nClosestBoids(boid, n):
    sorted_boids = sorted(boids, key=lambda other_boid: distance(boid, other_boid))
    return sorted_boids[1:n+1]

def sizeCanvas():
    global width, height
    size = (width, height)
    return pygame.display.set_mode(size)

def keepWithinBounds(boid):
    margin = 200
    turnFactor = 1
    if boid['x'] < margin:
        boid['dx'] += turnFactor
    if boid['x'] > width - margin:
        boid['dx'] -= turnFactor
    if boid['y'] < margin:
        boid['dy'] += turnFactor
    if boid['y'] > height - margin:
        boid['dy'] -= turnFactor

def flyTowardsCenter(boid):
    centeringFactor = 0.005
    neighbormin = 3
    centerX = 0
    centerY = 0
    numNeighbors = 0
    for otherBoid in boids:
        if distance(boid, otherBoid) < visualRange:
            centerX += otherBoid['x']
            centerY += otherBoid['y']
            numNeighbors += 1

    if numNeighbors <= neighbormin:
        for otherBoid in boids:
            if otherBoid in nClosestBoids(boid, boidneighbor):
                centerX += otherBoid['x']
                centerY += otherBoid['y']
                numNeighbors += 1
            if numNeighbors == neighbormin:
                break

    if numNeighbors:
        centerX /= numNeighbors
        centerY /= numNeighbors
        boid['dx'] += (centerX - boid['x']) * centeringFactor
        boid['dy'] += (centerY - boid['y']) * centeringFactor

def avoidOthers(boid):
    minDistance = 20
    avoidFactor = 0.05
    moveX = 0
    moveY = 0
    for otherBoid in boids:
        if otherBoid != boid:
            if distance(boid, otherBoid) < minDistance:
                moveX += boid['x'] - otherBoid['x']
                moveY += boid['y'] - otherBoid['y']
    boid['dx'] += moveX * avoidFactor
    boid['dy'] += moveY * avoidFactor

def matchVelocity(boid):
    matchingFactor = 0.05
    neighbormin = 3
    avgDX = 0
    avgDY = 0
    numNeighbors = 0
    for otherBoid in boids:
        if distance(boid, otherBoid) < visualRange:
            avgDX += otherBoid['dx']
            avgDY += otherBoid['dy']
            numNeighbors += 1

    if numNeighbors <= neighbormin:
        for otherBoid in boids:
            if otherBoid in nClosestBoids(boid, boidneighbor):
                avgDX += otherBoid['dx']
                avgDY += otherBoid['dy']
                numNeighbors += 1
            if numNeighbors == neighbormin:
                break

    if numNeighbors:
        avgDX /= numNeighbors
        avgDY /= numNeighbors
        boid['dx'] += (avgDX - boid['dx']) * matchingFactor
        boid['dy'] += (avgDY - boid['dy']) * matchingFactor

def limitSpeed(boid):
    speedLimit = 15
    speed = math.sqrt(boid['dx']**2 + boid['dy']**2)
    if speed > speedLimit:
        boid['dx'] = (boid['dx'] / speed) * speedLimit
        boid['dy'] = (boid['dy'] / speed) * speedLimit

def generate_flow_field(z_offset, scale, cols, rows):
    flow_field = np.zeros((cols, rows, 2))
    for y in range(rows):
        for x in range(cols):
            angle = noise.pnoise3(x * 0.1, y * 0.1, z_offset) * 2 * np.pi
            flow_field[x, y] = (np.cos(angle), np.sin(angle))
    return flow_field

def follow_flow_field(boid, flow_field, scale):
    col = int(boid['x'] / scale)
    row = int(boid['y'] / scale)
    if 0 <= col < flow_field.shape[0] and 0 <= row < flow_field.shape[1]:
        force = flow_field[col, row]
        boid['dx'] += force[0]
        boid['dy'] += force[1]

def draw_flow_field(screen, flow_field, scale):
    for y in range(flow_field.shape[1]):
        for x in range(flow_field.shape[0]):
            start_pos = (x * scale, y * scale)
            end_pos = (start_pos[0] + flow_field[x, y, 0] * scale, start_pos[1] + flow_field[x, y, 1] * scale)
            pygame.draw.line(screen, (100, 100, 100), start_pos, end_pos)

def emo_boid():
    emo_boid_num = 1
    emo_boid_lis = []
    for i in range (emo_boid_num):
        emo_boid_lis.append(random.choice(boids))
    for emo_boid in emo_boid_lis:
        def avoidOthers():
            minDistance = 20
            avoidFactor = 0.05
            moveX = 0
            moveY = 0

            for otherBoid in boids:
                if otherBoid != boid:
                    if distance(boid, otherBoid) < minDistance:
                        moveX += boid['x'] - otherBoid['x']
                        moveY += boid['y'] - otherBoid['y']

    boid['dx'] += moveX * avoidFactor
    boid['dy'] += moveY * avoidFactor    
    
def drawBoid(screen, boid, color):
    angle = math.atan2(boid['dy'], boid['dx'])
    arrow_shape = [
        (0, -15),   # tip of the arrow
        (10, -5),   # right wing
        (10, 0),    # right wing end
        (20, 0),    # tail end
        (10, 5),    # left wing end
        (10, 5),    # left wing
        (0, 15)     # tip of the arrow
    ]
    rotated_points = []
    for point in arrow_shape:
        x, y = point
        rotated_x = x * math.cos(angle) - y * math.sin(angle)
        rotated_y = x * math.sin(angle) + y * math.cos(angle)
        rotated_points.append((rotated_x + boid['x'], rotated_y + boid['y']))
    pygame.draw.polygon(screen, color, rotated_points)
    if DRAW_TRAIL:
        trail_length = len(boid['history'])
        for i, point in enumerate(boid['history']):
            alpha = int(255 * (1 - (i / trail_length)) * 0.5)
            faded_color = (color[0], color[1], color[2], alpha)
            s = pygame.Surface((4, 4), pygame.SRCALPHA)
            pygame.draw.circle(s, faded_color, (2, 2), 2)
            screen.blit(s, (int(point[0]), int(point[1])))

def animationLoop():
    global boids, z_offset
    screen.fill((0, 0, 0))
    scale = 20
    cols = width // scale
    rows = height // scale
    flow_field = generate_flow_field(z_offset, scale, cols, rows)
    draw_flow_field(screen, flow_field, scale)
    for boid in boids:
        follow_flow_field(boid, flow_field, scale)
        flyTowardsCenter(boid)
        avoidOthers(boid)
        matchVelocity(boid)
        limitSpeed(boid)
        keepWithinBounds(boid)
        boid['x'] += boid['dx']
        boid['y'] += boid['dy']
        boid['history'].append((boid['x'], boid['y']))
        boid['history'] = boid['history'][-30:]
        numNeighbors = sum(distance(boid, otherBoid) < visualRange for otherBoid in boids)
        brightness = min(255, numNeighbors * 10)
        if numNeighbors > 3:
            red = 255
            green = brightness
            blue = 0
        else:
            red = 0
            green = 0
            blue = 255
        color = (red, green, blue)
        drawBoid(screen, boid, color)
    pygame.display.flip()
    pygame.time.Clock().tick(60)
    z_offset += 0.01

pygame.init()
screen = sizeCanvas()
DRAW_TRAIL = True
z_offset = 0

initBoids()
while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
    animationLoop()


KeyboardInterrupt: 